In [1]:
%gui qt6

In [3]:
import sys
import socket
import datetime
import pandas as pd
from PyQt6.QtWidgets import (
    QApplication, QWidget, QPushButton, QVBoxLayout, QLabel, QHBoxLayout, QGridLayout, QTableWidget, 
    QTableWidgetItem, QSizePolicy, QSpacerItem, QFrame, QLineEdit, QTextEdit, QFileDialog, QButtonGroup, QCheckBox
)
from PyQt6.QtGui import QPixmap, QPalette, QBrush
from PyQt6.QtCore import Qt

SERVER_IP = "192.168.0.100"
SERVER_PORT = 5000
LOG_FILE_PATH = "received_logs.csv"
CRIMINAL_DETAILS_FILE = "criminal_details.csv"
LOGS_CSV_FILE = "logs.csv"

class CriminalDetectionGUI(QWidget):
    def __init__(self):
        super().__init__()

        # Window Setup
        self.setWindowTitle("Criminal Detection System")
        self.setGeometry(100, 100, 1200, 700)
        self.set_background_image(r"C:\Users\Santhana Krishnan\Desktop\project\bg.jpg")

        # Main Layout
        self.layout = QHBoxLayout(self)

        # Sidebar
        self.sidebar = QVBoxLayout()
        self.sidebar.setAlignment(Qt.AlignmentFlag.AlignTop | Qt.AlignmentFlag.AlignLeft)
        self.sidebar.setContentsMargins(20, 40, 10, 40)
        self.sidebar.setSpacing(20)

        self.sidebar_frame = QFrame()
        self.sidebar_frame.setLayout(self.sidebar)
        self.sidebar_frame.setStyleSheet("background: transparent;")
        self.sidebar_frame.setFixedWidth(250)

        # Sidebar Buttons
        self.button_texts = ["Home", "Access Logs", "Live Footage", "Enter Criminal Details"]
        self.buttons = []
        for text in self.button_texts:
            button = QPushButton(text, self)
            button.setStyleSheet(self.button_style(False))
            button.setCursor(Qt.CursorShape.PointingHandCursor)
            button.setSizePolicy(QSizePolicy.Policy.Expanding, QSizePolicy.Policy.Fixed)
            button.setFixedHeight(50)
            button.setMinimumWidth(200)
            button.clicked.connect(lambda checked, action=text: self.handle_option(action))
            self.sidebar.addWidget(button, alignment=Qt.AlignmentFlag.AlignLeft)
            self.buttons.append(button)

        self.sidebar.addItem(QSpacerItem(20, 40, QSizePolicy.Policy.Minimum, QSizePolicy.Policy.Expanding))
        self.layout.addWidget(self.sidebar_frame, alignment=Qt.AlignmentFlag.AlignLeft)
        self.setLayout(self.layout)

        # Access Logs Table
        self.log_table = QTableWidget()
        self.log_table.setColumnCount(3)
        self.log_table.setHorizontalHeaderLabels(["Date", "Time", "Event"])
        self.log_table.setVisible(False)
        self.log_table.setStyleSheet("""
            QTableWidget {
                background-color: rgba(0, 0, 0, 180); /* Slightly more transparent */
                color: white;
                gridline-color: rgba(255, 255, 255, 70); /* Slightly more visible gridlines */
                border: 1px solid rgba(255, 255, 255, 70);
            }

            QHeaderView::section {
                background-color: rgba(0, 0, 0, 220); /* Slightly darker for contrast */
                color: white;
                font-weight: bold;
                border: 1px solid rgba(255, 255, 255, 70);
                padding: 5px;
            }

            QTableWidget QTableCornerButton::section {
                background-color: rgba(0, 0, 0, 220); /* Matches header background */
                border: 1px solid rgba(255, 255, 255, 70);
            }

        """)


        # Criminal Details Form
        self.criminal_form = QWidget()
        self.form_layout = QGridLayout()
        label_style = "color: white; font-size: 14px; font-weight: bold;"
        input_style = "background-color: black; color: white; border: 1px solid white;"

        self.form_title = QLabel("Criminal Details Form")
        self.form_title.setStyleSheet("color: white; font-size: 18px; font-weight: bold;")
        self.form_layout.addWidget(self.form_title, 0, 0, 1, 4, alignment=Qt.AlignmentFlag.AlignCenter)
        self.save_status_label = QLabel("")
        self.save_status_label.setStyleSheet("color: lightgreen; font-weight: bold;")
        self.save_status_label.setVisible(False)  
        self.form_layout.addWidget(self.save_status_label, 1, 1, 1, 2)
        
        self.name_label = QLabel("Full Name:")
        self.name_label.setStyleSheet(label_style)
        self.name_input = QLineEdit()
        self.name_input.setStyleSheet(input_style)
        
        self.age_label = QLabel("Age:")
        self.age_label.setStyleSheet(label_style)
        self.age_input = QLineEdit()
        self.age_input.setStyleSheet(input_style)

        self.gender_label = QLabel("Gender:")
        self.gender_label.setStyleSheet(label_style)
        self.male_checkbox = QCheckBox("Male")
        self.female_checkbox = QCheckBox("Female")
        self.other_checkbox = QCheckBox("Other")
        checkbox_style = "color: white;"
        self.male_checkbox.setStyleSheet(checkbox_style)
        self.female_checkbox.setStyleSheet(checkbox_style)
        self.other_checkbox.setStyleSheet(checkbox_style)
        self.male_checkbox.toggled.connect(lambda: self.uncheck_others(self.male_checkbox))
        self.female_checkbox.toggled.connect(lambda: self.uncheck_others(self.female_checkbox))
        self.other_checkbox.toggled.connect(lambda: self.uncheck_others(self.other_checkbox))
        self.gender_layout = QHBoxLayout()
        self.gender_layout.addWidget(self.male_checkbox)
        self.gender_layout.addSpacing(20)  # Add spacing between checkboxes
        self.gender_layout.addWidget(self.female_checkbox)
        self.gender_layout.addSpacing(20)  # Add spacing between checkboxes
        self.gender_layout.addWidget(self.other_checkbox)

        self.dob_label = QLabel("Date of Birth:")
        self.dob_label.setStyleSheet(label_style)
        self.dob_input = QLineEdit()
        self.dob_input.setStyleSheet(input_style)
        
        self.num_label = QLabel("Mobile Number:")
        self.num_label.setStyleSheet(label_style)
        self.num_input = QLineEdit()
        self.num_input.setStyleSheet(input_style)

        self.id_num_label = QLabel("Identification Number:")
        self.id_num_label.setStyleSheet(label_style)
        self.id_num_input = QLineEdit()
        self.id_num_input.setStyleSheet(input_style)

        self.address_label = QLabel("Address:")
        self.address_label.setStyleSheet(label_style)
        self.address_input = QTextEdit()
        self.address_input.setStyleSheet(input_style)
        self.address_input.setFixedHeight(60)
        self.address_input.setFixedWidth(250)

        self.loc_label = QLabel("Location of Crime:")
        self.loc_label.setStyleSheet(label_style)
        self.loc_input = QLineEdit()
        self.loc_input.setStyleSheet(input_style)

        self.case_num_label = QLabel("Case Number:")
        self.case_num_label.setStyleSheet(label_style)
        self.case_num_input = QLineEdit()
        self.case_num_input.setStyleSheet(input_style)
        
        self.crime_desc_label = QLabel("Crime Committed:")
        self.crime_desc_label.setStyleSheet(label_style)
        self.crime_desc_input = QTextEdit()
        self.crime_desc_input.setStyleSheet(input_style)
        self.crime_desc_input.setFixedHeight(60)
        self.address_input.setFixedWidth(250)
        
        self.upload_button = QPushButton("Upload Image")
        self.upload_button.setCursor(Qt.CursorShape.PointingHandCursor)
        self.upload_button.clicked.connect(self.upload_image)

        self.save_button = QPushButton("Submit")
        self.save_button.setCursor(Qt.CursorShape.PointingHandCursor)
        self.save_button.clicked.connect(self.save_criminal_details)
        self.form_layout.addWidget(self.name_label, 2, 0)
        self.form_layout.addWidget(self.name_input, 2, 1)
        self.form_layout.addWidget(self.age_label, 3, 0)
        self.form_layout.addWidget(self.age_input, 3, 1)
        self.form_layout.addWidget(self.gender_label, 4, 0)
        self.form_layout.addLayout(self.gender_layout, 4, 1, 1, 3)
        self.form_layout.addWidget(self.dob_label, 5, 0)
        self.form_layout.addWidget(self.dob_input, 5, 1)
        self.form_layout.addWidget(self.num_label, 6, 0)
        self.form_layout.addWidget(self.num_input, 6, 1)
        self.form_layout.addWidget(self.id_num_label, 7, 0)
        self.form_layout.addWidget(self.id_num_input, 7, 1)
        self.form_layout.addWidget(self.address_label, 8, 0)
        self.form_layout.addWidget(self.address_input, 8, 1)
        self.form_layout.addWidget(self.crime_desc_label, 9, 0)
        self.form_layout.addWidget(self.crime_desc_input, 9, 1)
        self.form_layout.addWidget(self.case_num_label, 10, 0)
        self.form_layout.addWidget(self.case_num_input, 10, 1)
        self.form_layout.addWidget(self.loc_label, 11, 0)
        self.form_layout.addWidget(self.loc_input, 11, 1)
        self.image_preview = QLabel()
        self.image_preview.setFixedSize(200, 200)  
        self.image_preview.setStyleSheet("border: 2px solid white;")  
        self.image_preview.setAlignment(Qt.AlignmentFlag.AlignCenter)  # Center image
        self.form_layout.addWidget(self.image_preview, 12, 0, 1, 4, Qt.AlignmentFlag.AlignCenter)
        self.form_layout.addWidget(self.upload_button, 13, 0, 1, 2)
        self.form_layout.addWidget(self.save_button, 13, 2, 1, 2)
        
        self.criminal_form.setLayout(self.form_layout)
        self.criminal_form.setVisible(False)

        self.layout.addWidget(self.log_table, stretch=5)
        self.layout.addWidget(self.criminal_form, stretch=5)

        self.logs_loaded = False
        self.criminal_image_path = None  
        self.set_active_button(self.buttons[0])
        

    def set_background_image(self, image_path):
        """Set scaled background image"""
        palette = QPalette()
        pixmap = QPixmap(image_path)
        palette.setBrush(QPalette.ColorRole.Window, QBrush(pixmap.scaled(self.size(), Qt.AspectRatioMode.KeepAspectRatioByExpanding)))
        self.setPalette(palette)

    def resizeEvent(self, event):
        """Ensure background image scales with window resize"""
        self.set_background_image(r"C:\Users\alamu\Downloads\project\bg.jpg")
        super().resizeEvent(event)

    def sending(self, file_type, file_path, client_socket):
        """Sends the file type and CSV content to the server."""
        csv_data = self.read_csv(file_path)
        if not csv_data:
            print(f"Skipping {file_path}, as it's empty or cannot be read.")
            return
        
        # Send file type first
        client_socket.sendall(file_type.encode())
        ack = client_socket.recv(1024)  # Wait for ACK from server
    
        # Send CSV data
        client_socket.sendall(csv_data.encode())
        print(f"{file_type} data sent successfully.")
    
    def read_csv(self, file_path):
        """Reads a CSV file and returns its content as a string."""
        try:
            with open(file_path, "r") as file:
                return file.read()
        except Exception as e:
            print(f"Error reading {file_path}: {e}")
            return None
    
    def button_style(self, active):
        """Returns stylesheet for transparent buttons"""
        if active:
            return """
                color: #ADD8E6; 
                font-size: 18px; 
                background: transparent; 
                border: none; 
                text-align: left; 
                padding-left: 20px; 
                font-weight: bold;
            """
        else:
            return """
                color: #A9A9A9; 
                font-size: 18px; 
                background: transparent; 
                border: none; 
                text-align: left; 
                padding-left: 20px;
            """

    def set_active_button(self, button):
        """Handles button selection"""
        for btn in self.buttons:
            btn.setStyleSheet(self.button_style(False))
            btn.setText(btn.text().replace("• ", ""))
        
        button.setStyleSheet(self.button_style(True))
        button.setText("• " + button.text())

    def handle_option(self, option):
        """Handles button click events"""
        self.log_table.setVisible(False)
        self.criminal_form.setVisible(False)
        if option == "Access Logs":
            if not self.logs_loaded:
                self.fetch_and_display_logs()
                self.logs_loaded = True
            self.log_table.setVisible(True)
            self.criminal_form.setVisible(False)
            self.display_logs(LOGS_CSV_FILE)
        elif option == "Enter Criminal Details":
            self.log_table.setVisible(False)
            self.criminal_form.setVisible(True)
        else:
            self.log_table.setVisible(False)
            self.criminal_form.setVisible(False)

        self.set_active_button(next(btn for btn in self.buttons if btn.text().strip("• ") == option))

    def uncheck_others(self, selected_checkbox):
        """ Unchecks all other checkboxes except the selected one. """
        for checkbox in [self.male_checkbox, self.female_checkbox, self.other_checkbox]:
            if checkbox != selected_checkbox:
                checkbox.setChecked(False)
        
    def fetch_and_display_logs(self):
        """Fetch logs from server and update the UI"""
        current_time = datetime.datetime.now().strftime("%Y-%m-%d,%H:%M:%S")
        log_data = f"{current_time},Access Log Requested\n"
        with open(LOGS_CSV_FILE, "a") as file:
            file.write(log_data) 
             
        try:
            with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as client_socket:
                client_socket.connect((SERVER_IP, SERVER_PORT))
                self.sending("LOG", LOGS_CSV_FILE, client_socket)

                with open(LOG_FILE_PATH, "wb") as file:
                    while True:
                        data = client_socket.recv(1024)
                        if not data:
                            break
                        file.write(data)

            self.display_logs(LOGS_CSV_FILE)
        except Exception as e:
            print(f"Error fetching logs: {e}")

    def display_logs(self, csv_file):
        """Display logs in table format with adjusted column widths and black scrollbar"""
        df = pd.read_csv(csv_file)
        self.log_table.setRowCount(df.shape[0])
        self.log_table.setColumnCount(df.shape[1])

        for row in range(df.shape[0]):
            for col in range(df.shape[1]):
                item = QTableWidgetItem(str(df.iat[row, col]))
                item.setTextAlignment(Qt.AlignmentFlag.AlignCenter)
                self.log_table.setItem(row, col, item)

        # Set specific column widths
        self.log_table.setColumnWidth(0, 150)  # Date column
        self.log_table.setColumnWidth(1, 120)  # Time column
        self.log_table.setColumnWidth(2, self.width() - 300)  # Event column fills remaining space

        # Enable horizontal scrolling for better layout
        self.log_table.horizontalHeader().setStretchLastSection(True)

        self.log_table.setVisible(True)

    def get_selected_gender(self):
        """Returns the selected gender label."""
        if self.male_checkbox.isChecked():
            return "Male"
        elif self.female_checkbox.isChecked():
            return "Female"
        elif self.other_checkbox.isChecked():
            return "Other"
        return "Not Specified"
    
    def upload_image(self):
        """Allows user to select an image and previews it in the label"""
        file_dialog = QFileDialog()
        file_path, _ = file_dialog.getOpenFileName(self, "Select Image", "", "Images (*.png *.jpg *.jpeg *.bmp)")
    
        if file_path:
            self.criminal_image_path = file_path  # Store file path
            pixmap = QPixmap(file_path)  # Load the image
            self.image_preview.setPixmap(pixmap.scaled(200, 200, Qt.AspectRatioMode.KeepAspectRatio))  # Resize to fit
    

    def save_criminal_details(self):
        """Saves entered criminal details to a CSV file"""
        name = self.name_input.text()
        age = self.age_input.text()
        gender = self.get_selected_gender()
        dob = self.dob_input.text()
        num = self.num_input.text()
        id_num = self.id_num_input.text()
        address = self.address_input.toPlainText()
        crime_desc = self.crime_desc_input.toPlainText()
        case_num = self.case_num_input.text()
        loc = self.loc_input.text()

        with open(CRIMINAL_DETAILS_FILE, "a") as file:
            file.write(f"{name},{age},{gender},{dob},{num},{id_num},{address},{crime_desc},{case_num},{loc},{self.criminal_image_path}\n")

        self.save_status_label.setText("Criminal Details Saved!")
        self.save_status_label.setStyleSheet("color: lightgreen; font-weight: bold;")
        self.save_status_label.setVisible(True)
        self.send_csv_to_server(CRIMINAL_DETAILS_FILE)
        # Reset all fields to empty
        self.name_input.clear()
        self.age_input.clear()
        self.dob_input.clear()
        self.num_input.clear()
        self.id_num_input.clear()
        self.address_input.clear()
        self.crime_desc_input.clear()
        self.case_num_input.clear()
        self.loc_input.clear()
        self.male_checkbox.setChecked(False)
        self.female_checkbox.setChecked(False)
        self.other_checkbox.setChecked(False)
        self.image_preview.clear()
        self.criminal_image_path = None
        self.image_preview.setStyleSheet("border: 2px solid white; color: gray; font-size: 12px;")
        

    def send_csv_to_server(self, file_path):
        """Sends the CSV file to the server via socket"""
        try:
            with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as client_socket:
                client_socket.connect((SERVER_IP, SERVER_PORT))
                self.sending("CRIMINAL", CRIMINAL_DETAILS_FILE, client_socket)
            print("File sent to server successfully!")
        except Exception as e:
            print(f"Error sending file: {e}")
        
if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = CriminalDetectionGUI()
    window.show()
    sys.exit(app.exec())


LOG data sent successfully.


SystemExit: 0

C:\Users\Santhana Krishnan\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
